In [6]:
# Copyright (c) Microsoft Corporation.
# Licensed under the MIT License.

"""A sample run of the CyberBattle simulation"""

from typing import cast
import gymnasium as gym
import numpy as np
import logging
import sys
import copy
from cyberbattle._env.cyberbattle_env import CyberBattleEnv

In [ ]:

"""Entry point if called as an executable"""

# root = logging.getLogger()
# root.setLevel(logging.DEBUG)

# handler = logging.StreamHandler(sys.stdout)
# handler.setLevel(logging.DEBUG)
# formatter = logging.Formatter("%(asctime)s - %(name)s - %(levelname)s - %(message)s")
# handler.setFormatter(formatter)
# root.addHandler(handler)

# env = cast(CyberBattleEnv, gym.make("CyberBattleChain-v0"))
env = cast(CyberBattleEnv, gym.make("CyberBattleToyCtf-v0"))


In [8]:
def estimate_position_value_with_random_actions(env, max_actions=50):
    total_reward = 0
    already_succeded_actions = []
    action_mask = env.compute_action_mask()
    for t in range(max_actions):
        # env.render()
        # print(f"can connect= {action_mask['connect'].any()}")
        # print(f"valid actions: {env.get_all_valid_actions()}")
        # # sample a valid action
        action = env.sample_valid_action()
        while (not env.apply_mask(action, action_mask)) and not (action in already_succeded_actions):
            action = env.sample_valid_action()

        observation, reward, done, truncated, info = env.step(action)

        if reward > 0:
            already_succeded_actions.append(action)

        action_mask = observation["action_mask"]
        total_reward = total_reward + reward

        if truncated and t != 0:
            return total_reward / t
        if done and t != 0:
            return total_reward / t
        if (done or truncated) and t == 0:
            return 0
    return total_reward / max_actions

In [9]:
def canonicalize_action(action):
    return {k: tuple(v.tolist()) for k, v in action.items()}


def greedy_search(env, already_succeded_actions, depth=1):
    if depth == 0:
        return None, 0

    best_action = None
    best_pos_val = float("-inf")
    score = float("-inf")
    # print(f"All valid actions in depth {depth} {env.get_all_valid_actions()}")

    print(already_succeded_actions)
    for action in env.get_all_valid_actions():
        if any(canonicalize_action(action) == canonicalize_action(a)
            for a in already_succeded_actions):
                continue

        env_copy = copy.deepcopy(env)
        env_copy.np_random = None
        _, reward, done, truncated, _ = env_copy.step(action)

        if not done and not truncated:
            pos_val = estimate_position_value_with_random_actions(env_copy, 15)
            _, future_reward = greedy_search(env_copy, already_succeded_actions, depth-1)
            reward += future_reward

            score = reward + future_reward + pos_val

        if score > best_pos_val and action:
            best_pos_val = score
            best_action = action

    return best_action, best_pos_val

In [10]:
obs, _ = env.reset()
total_reward = 0

done = False
truncated = False

already_succeded_actions = []
while not done and not truncated:
    best_action, expected_reward = greedy_search(env, already_succeded_actions, depth=1)
    print("Chosen action:", best_action, "| expected cumulative reward:", expected_reward)

    obs, reward, done, truncated, info = env.step(best_action)
    env.render()

    if reward > 0:
        already_succeded_actions.append(best_action)

    total_reward += reward

print("Total reward collected:", total_reward)
env.close()

/opt/miniconda/envs/cybersim/lib/python3.10/site-packages/gymnasium/utils/passive_env_checker.py:127: UserWarning:

WARN: The obs returned by the `reset()` method should be an int or np.int64, actual type: <class 'numpy.int32'>

/opt/miniconda/envs/cybersim/lib/python3.10/site-packages/gymnasium/core.py:311: UserWarning:

WARN: env.get_all_valid_actions to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.get_all_valid_actions` for environment variables or `env.get_wrapper_attr('get_all_valid_actions')` that will search the reminding wrappers.



[]


/opt/miniconda/envs/cybersim/lib/python3.10/site-packages/gymnasium/utils/passive_env_checker.py:127: UserWarning:

WARN: The obs returned by the `step()` method should be an int or np.int64, actual type: <class 'numpy.int32'>

/opt/miniconda/envs/cybersim/lib/python3.10/site-packages/gymnasium/core.py:311: UserWarning:

WARN: env.compute_action_mask to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.compute_action_mask` for environment variables or `env.get_wrapper_attr('compute_action_mask')` that will search the reminding wrappers.

/opt/miniconda/envs/cybersim/lib/python3.10/site-packages/gymnasium/core.py:311: UserWarning:

WARN: env.sample_valid_action to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.sample_valid_action` for environment variables or `env.get_wrapper_attr('sample_valid_action')` that will search the reminding wra

Chosen action: {'local_vulnerability': array([0, 1], dtype=int32)} | expected cumulative reward: 14.4


/opt/miniconda/envs/cybersim/lib/python3.10/site-packages/gymnasium/utils/passive_env_checker.py:335: UserWarning:

WARN: No render fps was declared in the environment (env.metadata['render_fps'] is None or not defined), rendering may occur at inconsistent fps.



...

Chosen action: {'connect': array([1, 4, 5, 0], dtype=int32)} | expected cumulative reward: 2503.0


,status,properties,local_attacks,remote_attacks
id,,,,
start,owned,[],[ScanExplorerRecentFiles],[]
1_LinuxNode,owned,"[MySql, Ubuntu, nginx/1.10.3]","[SudoAttempt, ScanExplorerRecentFiles, CrackKe...","[ProbeWindows, ProbeLinux]"
2_WindowsNode,owned,"[Windows, Win10, Win10Patched]","[SudoAttempt, CrackKeepPassX, CrackKeepPass, S...","[ProbeWindows, ProbeLinux]"
3_LinuxNode,owned,"[MySql, Ubuntu, nginx/1.10.3]","[SudoAttempt, ScanExplorerRecentFiles, CrackKe...","[ProbeWindows, ProbeLinux]"
4_WindowsNode,owned,"[Windows, Win10, Win10Patched]","[SudoAttempt, CrackKeepPassX, CrackKeepPass, S...","[ProbeWindows, ProbeLinux]"
5_LinuxNode,discovered,NaN,None,[]


[{'local_vulnerability': array([0, 1], dtype=int32)}, {'connect': array([0, 1, 2, 0], dtype=int32)}, {'local_vulnerability': array([1, 3], dtype=int32)}, {'connect': array([0, 2, 3, 1], dtype=int32)}, {'local_vulnerability': array([2, 4], dtype=int32)}, {'connect': array([0, 3, 2, 2], dtype=int32)}, {'local_vulnerability': array([3, 3], dtype=int32)}, {'connect': array([0, 4, 3, 3], dtype=int32)}, {'local_vulnerability': array([4, 4], dtype=int32)}]


/opt/miniconda/envs/cybersim/lib/python3.10/site-packages/gymnasium/core.py:311: UserWarning:

WARN: env.get_all_valid_actions to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.get_all_valid_actions` for environment variables or `env.get_wrapper_attr('get_all_valid_actions')` that will search the reminding wrappers.

/opt/miniconda/envs/cybersim/lib/python3.10/site-packages/gymnasium/core.py:311: UserWarning:

WARN: env.compute_action_mask to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.compute_action_mask` for environment variables or `env.get_wrapper_attr('compute_action_mask')` that will search the reminding wrappers.

/opt/miniconda/envs/cybersim/lib/python3.10/site-packages/gymnasium/core.py:311: UserWarning:

WARN: env.sample_valid_action to get variables from other wrappers is deprecated and will be removed in v1.0, to get th

KeyboardInterrupt: 